Credit where credit is due: this is based on  https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp.

Loading the model will take a while so please be patient

In [1]:
# clean up after failed efforts if necessary
!rm -rf llama.cpp
# we need no CUDA for CPU; this flag has changed since the initial release
!CMAKE_ARGS="-DLLAMA_CUDA=off" pip install llama-cpp-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=3658483 sha256=241ccac6db2b167cd3e0f579c6a5f568a13c3f9ae8e01c960bda65b81aa7e7de
  Stored in directory: /root/.cache/pip/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python


In [2]:
import arrow
from llama_cpp import Llama

time_load = arrow.now()
llm = Llama(
    model_path="/kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf",
    n_ctx=4096,      # Max tokens for in + out
    n_threads=4,     # CPU cores used
    n_gpu_layers=-1,  # Load all layers into VRAM of the GPU
)
print("Model load time: {:5.4f} seconds".format((arrow.now() - time_load).total_seconds()))

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 lla

Model load time: 41.1672 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'tokenizer.chat_template': "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}", 'tokenizer.ggml.eos_token_id': '128009', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'general.architecture': 'llama', 'llama.rope.freq_base': '500000.000000', 'llama.context_length': '8192', 'general.name': '.', 'llama.vocab_size': '128256', 'general.file_type': '17', 'llama.embedding_length': '4096', 'lla

In [3]:
def ask(query: str, max_tokens: int):
    time_ask = arrow.now()
    output = llm("Q: {} A: ".format(query), max_tokens=max_tokens, stop=["Q:", "\n"], echo=False)
    return [output['choices'][index]['text'] for index in range(len(output['choices']))], (arrow.now() - time_ask).total_seconds()

query_cities = 'What are the names of three European capital cities?'
result_cities, time_cities = ask(query=query_cities, max_tokens=40)
print(query_cities)
print(result_cities)
print('time: {:5.4f}'.format(time_cities))


llama_print_timings:        load time =    3931.79 ms
llama_print_timings:      sample time =      29.55 ms /    14 runs   (    2.11 ms per token,   473.74 tokens per second)
llama_print_timings: prompt eval time =    3931.41 ms /    16 tokens (  245.71 ms per token,     4.07 tokens per second)
llama_print_timings:        eval time =    4166.83 ms /    13 runs   (  320.53 ms per token,     3.12 tokens per second)
llama_print_timings:       total time =    8149.65 ms /    29 tokens


What are the names of three European capital cities?
['1. Paris, 2. Berlin, 3. Rome']
time: 8.1607


In [4]:
query_catcher = 'What can you tell me about the J. D. Salinger novel The Catcher in the Rye?'
result_catcher, time_catcher = ask(query=query_catcher, max_tokens=1000)
print(query_catcher)
print(result_catcher)
print('time: {:5.4f}'.format(time_catcher))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3931.79 ms
llama_print_timings:      sample time =     130.92 ms /    59 runs   (    2.22 ms per token,   450.65 tokens per second)
llama_print_timings: prompt eval time =    6548.11 ms /    24 tokens (  272.84 ms per token,     3.67 tokens per second)
llama_print_timings:        eval time =   18721.12 ms /    58 runs   (  322.78 ms per token,     3.10 tokens per second)
llama_print_timings:       total time =   25485.50 ms /    82 tokens


What can you tell me about the J. D. Salinger novel The Catcher in the Rye?
['\xa0The Catcher in the Rye is a classic coming-of-age novel published in 1951 by Jerome David Salinger, commonly known as J.D. Salinger. The story follows the protagonist, Holden Caulfield, a disillusioned teenager who has been expelled from several prestigious boarding schools.']
time: 25.4930


In [5]:
r, t = ask(query='How do I drive from Washington DC to New York City?', max_tokens=3000)
print(r)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3931.79 ms
llama_print_timings:      sample time =      80.62 ms /    36 runs   (    2.24 ms per token,   446.56 tokens per second)
llama_print_timings: prompt eval time =    3674.96 ms /    15 tokens (  245.00 ms per token,     4.08 tokens per second)
llama_print_timings:        eval time =   12475.31 ms /    35 runs   (  356.44 ms per token,     2.81 tokens per second)
llama_print_timings:       total time =   16277.16 ms /    50 tokens


['\xa0The driving distance from Washington D.C. to New York City is approximately 225 miles (362 kilometers) and typically takes around 4-5 hours depending on traffic conditions.']


In [6]:
r0, t0 = ask(query='What would I need to do to build an edible bicycle?', max_tokens=3000)
print(r0)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3931.79 ms
llama_print_timings:      sample time =      64.14 ms /    30 runs   (    2.14 ms per token,   467.73 tokens per second)
llama_print_timings: prompt eval time =    3675.62 ms /    15 tokens (  245.04 ms per token,     4.08 tokens per second)
llama_print_timings:        eval time =    9337.66 ms /    29 runs   (  321.99 ms per token,     3.11 tokens per second)
llama_print_timings:       total time =   13112.47 ms /    44 tokens


['\xa0Building an edible bicycle is a creative and challenging project that requires careful planning, design, and execution. Here are the general steps you could follow:']
